In [2]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import os
from datetime import datetime
import pandas as pd
import plotly.express as px

from download import download


mpl.rcParams['figure.figsize'] = (10, 8)
mpl.rcParams['axes.grid'] = False


In [8]:
df = pd.read_csv('filtered_nn_data.csv')

In [9]:
df.head()

,date,year,uv,buyer,gmv,orders,cr,day_of_week,weekday,day_type,day_binary,campaign_info,campaign_type
0,2021-01-01,2021,96281,3595,66052,4331,0.037,4,1,Weekday,100,non campaign,non-campaign
1,2021-01-02,2021,93620,3852,68483,4599,0.041,5,0,Weekend,101,non campaign,non-campaign
2,2021-01-03,2021,98196,4402,75556,5254,0.045,6,0,Weekend,110,non campaign,non-campaign
3,2021-01-04,2021,108903,4374,75432,5292,0.040,0,1,Weekday,0,non campaign,non-campaign
4,2021-01-05,2021,123715,4486,74678,5518,0.036,1,1,Weekday,1,non campaign,non-campaign


In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 748 entries, 0 to 747
Data columns (total 13 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   date           748 non-null    object 
 1   year           748 non-null    int64  
 2   uv             748 non-null    int64  
 3   buyer          748 non-null    int64  
 4   gmv            748 non-null    int64  
 5   orders         748 non-null    int64  
 6   cr             748 non-null    float64
 7   day_of_week    748 non-null    int64  
 8   weekday        748 non-null    int64  
 9   day_type       748 non-null    object 
 10  day_binary     748 non-null    int64  
 11  campaign_info  748 non-null    object 
 12  campaign_type  748 non-null    object 
dtypes: float64(1), int64(8), object(4)
memory usage: 76.1+ KB
